In [181]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf # use it to train the module
from sklearn.preprocessing import LabelEncoder # encoder to make the dataset as number
from sklearn.preprocessing import OneHotEncoder # encoder to make the dataset as one hot
from sklearn.utils import shuffle # use for shuffle the data
from sklearn.model_selection import train_test_split # split the data set
from sklearn.preprocessing import normalize # use to normalize the data set


# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# get the training se
df_origin = pd.read_csv("../input/train.csv")
train_df = df_origin.copy()
train_df.head()

['gender_submission.csv', 'test.csv', 'train.csv']


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [182]:
# # split the data set to X_train and Y_train
# Y_train = train_df['Survived']
# X_train = train_df.drop(['Survived'], axis=1)
# print('X_train: ', X_train.shape)
# print('Y_train: ', X_train.shape)
# print('-------------')
# X_train.head()

In [183]:
print(train_df.isnull().any())

# get the data distribution
train_df.describe()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
dtype: bool


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


# Set the sex to binary value

In [184]:
print(train_df['Sex'].describe())
#print(np.bincount(X_train['Sex']))

# set the sex to binary value
# le_sex = LabelEncoder().fit(train_df['Sex'])
# Sex_label = le_sex.transform(train_df['Sex']) # don't give the result to dataframe directely, we need to one hot encode later
# # Set the sex to on-hot
# #    545                     "Reshape your data either using array.reshape(-1, 1) if "
# #     546                     "your data has a single feature or array.reshape(1, -1) "
# # --> 547                     "if it contains a single sample.".format(array))
# # categories='auto' is follow the warning instruction
# OH_sex = OneHotEncoder(sparse=False, categories='auto')
# Sex_label = OH_sex.fit_transform(Sex_label.reshape(-1, 1 ))

# print(Sex_label)
# #train_df['Sex_Lable'] = Sex_label
# train_df['Sex_Lable'] = np.NaN
# train_df['Sex_Lable'] = Sex_label

# print(train_df['Sex_Lable'].iloc[0])
# print(train_df['Sex_Lable'].iloc[0].dtype)
############ old method #########################
# X_train['Sex_binary'] = X_train['Sex'] == 'male'
# #X_train['Sex_binary'].head()
# X_train['Sex_binary'] = X_train['Sex_binary'].astype('int')
# #X_train['Sex'].head()

# print(X_train['Sex_binary'].describe())
# # check the covariance
# print(np.cov(X_train['Sex_binary'], train_df.Survived))
sex_df = pd.get_dummies(df_origin['Sex'], columns='Sex', prefix='Sex')
train_df = pd.concat([train_df, sex_df], axis=1, sort=False)
train_df = train_df.drop(['Sex'], axis=1)
#train_df.head()

count      891
unique       2
top       male
freq       577
Name: Sex, dtype: object


In [185]:
# feature engineering on Embarked

# deal with the NaN or just use the NaN as a feature
# print(df_origin['Embarked'].isnull().any())
# Embarked_nan_mask = df_origin['Embarked'].isnull()
# #Try using .loc[row_indexer,col_indexer] = value instead
# Embarked_lable = train_df['Embarked'].copy()
# Embarked_lable[Embarked_nan_mask] = 'N'
# train_df['Embarked_lable'] = Embarked_lable
# print(train_df['Embarked_lable'].isnull().any())
# print(train_df[Embarked_nan_mask])
eb_df = pd.get_dummies(df_origin['Embarked'], columns='Embarked', dummy_na=True, prefix='Embarked')
train_df = pd.concat([train_df, eb_df], axis=1, sort=False)
train_df = train_df.drop(['Embarked'], axis=1)


#one hot encode
# le_Eb = LabelEncoder().fit(train_df['Embarked_lable'])
# train_df['Embarked_lable'] = le_Eb.transform(train_df['Embarked_lable']) # don't give the result to dataframe directely, we need to one hot encode later
# #print(train_df['Embarked_lable'])

# # categories='auto' is follow the warning instruction
# #  sklearn 的新版本中，OneHotEncoder 的输入必须是 2-D array，而 testdata.age 返回的 Series 本质上是 1-D array，所以要改成
# # 
# a = OneHotEncoder(sparse=False, categories='auto').fit_transform(train_df[['Embarked_lable']])
# print(a)


#print(train_df['Embarked_lable'])
#train_df['Sex_Lable'] = Sex_label
#train_df['Embarked_lable'] = Eb_label


In [186]:
# deal with the Cabin
print(df_origin['Cabin'].describe())
print(df_origin['Cabin'][0], 'is ', type(df_origin['Cabin'][0]))
print(df_origin['Cabin'][1], 'is ', type(df_origin['Cabin'][1]))

# the nan in cabin type is float
# cabin_nan_mask = df_origin['Cabin'].isna()
# print(cabin_nan_mask)
# df_origin[cabin_nan_mask]

def get_cabin_level(x):
    #print(x)
    if x == 0:
        return x
    else:
        return x[0]

# make the NaN to 0
train_df['Cabin_label']= train_df['Cabin'].fillna(0)
    
# # modify to cabin level, such as C23 to level C
train_df['Cabin_label'] = train_df['Cabin_label'].map(get_cabin_level)
#train_df = train_df.drop(['Cabin'], axis=1)

cb_df = pd.get_dummies(train_df['Cabin_label'], columns='Cabin_label', dummy_na=True, prefix='Cabin')
train_df = pd.concat([train_df, cb_df], axis=1, sort=False)
train_df = train_df.drop(['Cabin_label'], axis=1)  
train_df = train_df.drop(['Cabin'], axis=1)

count             204
unique            147
top       C23 C25 C27
freq                4
Name: Cabin, dtype: object
nan is  <class 'float'>
C85 is  <class 'str'>


In [187]:
# deal with the age
train_df['Age_label']= np.floor(train_df['Age'].fillna(np.mean(df_origin['Age'])))
train_df = train_df.drop(['Age'], axis=1)   

In [188]:
# drop unneed column
train_df = train_df.drop(['Name', 'Ticket', 'PassengerId'], axis=1)

# shuffle the data
train_df = shuffle(train_df)

In [190]:
# normalize the fare
f_temp = train_df['Fare'].fillna(np.mean(df_origin['Fare']))
fare_max = np.max(f_temp)
fare_min = np.min(f_temp)
train_df['Fare'] = f_temp.apply(lambda x: (x - fare_min) / (fare_max - fare_min))

In [196]:
Y = train_df['Survived'].values
X = train_df.drop(['Survived'], axis=1).values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
m_feature = X_train.shape[1]
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 712
number of test examples = 179
X_train shape: (712, 21)
Y_train shape: (712,)
X_test shape: (179, 21)
Y_test shape: (179,)


In [198]:
def add_layer(inputs, in_size, out_size, activation_function = None):
    Weights = tf.Variable(tf.random_normal([in_size, out_size]))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
    
    Wx_plus_b = tf.matmul(inputs, Weights) + biases
    
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
        
    return outputs

xs = tf.placeholder(tf.float32,  [None, m_feature])
ys = tf.placeholder(tf.int32,  [None])
# add relu here
logits = add_layer(xs, m_feature, 2, activation_function=tf.nn.sigmoid)
predict = tf.arg_max(logits, 1)

loss = tf.losses.sparse_softmax_cross_entropy(logits=logits,labels=ys)
loss = tf.reduce_mean(loss)
optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)

_, acc_op = tf.metrics.accuracy(labels=ys,predictions=predict)

In [201]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())

    for epoch in range(1000): # 训练次数
        loss_value,_ ,acc_value= sess.run([loss,optimizer,acc_op],feed_dict={xs:X_train,ys:Y_train})
        print('loss = {}, acc = {}'.format(loss_value,acc_value))
    acc_value = sess.run([acc_op],feed_dict={xs:X_test,ys:Y_test}) # 验证集的精度，这里验证集和
    print('val acc = {}'.format(acc_value))


loss = 0.9234902262687683, acc = 0.3834269642829895
loss = 0.9231921434402466, acc = 0.3834269642829895
loss = 0.9229004383087158, acc = 0.3834269642829895
loss = 0.9226197004318237, acc = 0.3834269642829895
loss = 0.9223536849021912, acc = 0.3834269642829895
loss = 0.9221059679985046, acc = 0.3834269642829895
loss = 0.9218785762786865, acc = 0.3834269642829895
loss = 0.9216728806495667, acc = 0.3832513988018036
loss = 0.9214887022972107, acc = 0.38311484456062317
loss = 0.9213253259658813, acc = 0.38300561904907227
loss = 0.9211814999580383, acc = 0.38291624188423157
loss = 0.9210549592971802, acc = 0.3828417658805847
loss = 0.9209439754486084, acc = 0.38288676738739014
loss = 0.920846164226532, acc = 0.3829253613948822
loss = 0.920759379863739, acc = 0.3829587996006012
loss = 0.9206820726394653, acc = 0.3829880654811859
loss = 0.9206116795539856, acc = 0.38301387429237366
loss = 0.9205464720726013, acc = 0.3830368220806122
loss = 0.920484721660614, acc = 0.3830573558807373
loss = 0.9

loss = 0.6701094508171082, acc = 0.38622820377349854
loss = 0.6698950529098511, acc = 0.3862205147743225
loss = 0.6696828603744507, acc = 0.38621294498443604
loss = 0.6694731116294861, acc = 0.3861977756023407
loss = 0.6692658066749573, acc = 0.3861752152442932
loss = 0.6690608263015747, acc = 0.38615289330482483
loss = 0.6688587069511414, acc = 0.38613080978393555
loss = 0.6686593890190125, acc = 0.38610896468162537
loss = 0.6684629917144775, acc = 0.3860873281955719
loss = 0.6682690978050232, acc = 0.38606593012809753
loss = 0.6680780649185181, acc = 0.38604477047920227
loss = 0.6678896546363831, acc = 0.3860238194465637
loss = 0.6677036285400391, acc = 0.3860030770301819
loss = 0.6675199270248413, acc = 0.38597533106803894
loss = 0.6673386096954346, acc = 0.38594064116477966
loss = 0.6671594977378845, acc = 0.38589200377464294
loss = 0.6669824123382568, acc = 0.38583672046661377
loss = 0.6668068766593933, acc = 0.3857748806476593
loss = 0.6666334271430969, acc = 0.38572779297828674


loss = 0.6342900395393372, acc = 0.3833567500114441
loss = 0.6342480182647705, acc = 0.38336798548698425
loss = 0.6342067122459412, acc = 0.38337916135787964
loss = 0.6341661810874939, acc = 0.38339030742645264
loss = 0.6341264843940735, acc = 0.38340136408805847
loss = 0.6340873837471008, acc = 0.38341236114501953
loss = 0.6340490579605103, acc = 0.3834233283996582
loss = 0.6340113878250122, acc = 0.3834342360496521
loss = 0.6339743137359619, acc = 0.38344505429267883
loss = 0.6339378952980042, acc = 0.3834558427333832
loss = 0.6339018940925598, acc = 0.38346657156944275
loss = 0.6338667273521423, acc = 0.38347724080085754
loss = 0.6338320970535278, acc = 0.38348788022994995
loss = 0.633797824382782, acc = 0.3834984302520752
loss = 0.6337641477584839, acc = 0.38350895047187805
loss = 0.633730947971344, acc = 0.38351941108703613
loss = 0.6336984634399414, acc = 0.38352981209754944
loss = 0.6336662769317627, acc = 0.38354018330574036
loss = 0.6336347460746765, acc = 0.3835504651069641
l

loss = 0.6307392120361328, acc = 0.3843759596347809
loss = 0.6307260990142822, acc = 0.3843792974948883
loss = 0.6307128071784973, acc = 0.3843826353549957
loss = 0.6306995153427124, acc = 0.38438594341278076
loss = 0.6306864023208618, acc = 0.3843892514705658
loss = 0.6306732296943665, acc = 0.38439255952835083
loss = 0.6306599974632263, acc = 0.3843958377838135
loss = 0.6306470632553101, acc = 0.3843991160392761
loss = 0.630634069442749, acc = 0.3844023644924164
loss = 0.6306209564208984, acc = 0.38440564274787903
loss = 0.6306079626083374, acc = 0.3844088613986969
loss = 0.6305948495864868, acc = 0.38441210985183716
loss = 0.6305819153785706, acc = 0.38441532850265503
loss = 0.6305689215660095, acc = 0.3844185173511505
loss = 0.6305559873580933, acc = 0.384421706199646
loss = 0.6305431127548218, acc = 0.3844248950481415
loss = 0.6305304765701294, acc = 0.3844280540943146
loss = 0.6305176019668579, acc = 0.38443124294281006
loss = 0.6305047869682312, acc = 0.38443437218666077
loss = 

loss = 0.6285372376441956, acc = 0.3849344551563263
loss = 0.6285273432731628, acc = 0.3849380612373352
loss = 0.6285174489021301, acc = 0.3849416673183441
loss = 0.6285076141357422, acc = 0.38494524359703064
loss = 0.628497838973999, acc = 0.38494881987571716
loss = 0.6284881830215454, acc = 0.3849523663520813
loss = 0.6284784078598022, acc = 0.3849559426307678
loss = 0.6284686923027039, acc = 0.38495948910713196
loss = 0.6284589171409607, acc = 0.3849630057811737
loss = 0.6284493207931519, acc = 0.38496655225753784
loss = 0.6284396648406982, acc = 0.3849700689315796
loss = 0.6284298300743103, acc = 0.38497358560562134
loss = 0.628420352935791, acc = 0.3849770724773407
loss = 0.6284106969833374, acc = 0.38498055934906006
loss = 0.628400981426239, acc = 0.3849840462207794
loss = 0.628391444683075, acc = 0.3849875032901764
loss = 0.6283817887306213, acc = 0.38499099016189575
loss = 0.6283721923828125, acc = 0.3849944472312927
loss = 0.6283626556396484, acc = 0.3849978744983673
loss = 0.

loss = 0.6271085143089294, acc = 0.38541513681411743
loss = 0.6271007657051086, acc = 0.38541755080223083
loss = 0.6270929574966431, acc = 0.3854199945926666
loss = 0.6270850300788879, acc = 0.38542240858078003
loss = 0.6270771026611328, acc = 0.3854248523712158
loss = 0.6270692944526672, acc = 0.3854272663593292
loss = 0.6270614862442017, acc = 0.3854296803474426
loss = 0.6270537376403809, acc = 0.38543206453323364
loss = 0.6270459890365601, acc = 0.38543447852134705
loss = 0.6270381808280945, acc = 0.38543686270713806
loss = 0.6270304322242737, acc = 0.38543927669525146
loss = 0.6270225644111633, acc = 0.3854416608810425
loss = 0.6270148158073425, acc = 0.3854440152645111
loss = 0.627007246017456, acc = 0.3854463994503021
loss = 0.6269993185997009, acc = 0.38544878363609314
loss = 0.6269918084144592, acc = 0.38545113801956177
loss = 0.6269840002059937, acc = 0.3854534924030304
loss = 0.6269763708114624, acc = 0.385455846786499
loss = 0.6269686222076416, acc = 0.38545820116996765
loss

In [180]:
train_df.head(10)

,Survived,Pclass,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,...,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_nan,Age_label
267,1,3,1,0,7.7750,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,25.0
467,0,1,0,0,26.5500,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,56.0
352,0,3,1,1,7.2292,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,15.0
494,0,3,0,0,8.0500,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,21.0
361,0,2,1,0,27.7208,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,29.0
22,1,3,0,0,8.0292,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,15.0
93,0,3,1,2,20.5750,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,26.0
508,0,3,0,0,22.5250,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,28.0
80,0,3,0,0,9.0000,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,22.0
493,0,1,0,0,49.5042,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,71.0
